In [23]:
# import torch, tokenizer and bert-base-uncased model
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [24]:
# import datasets lib and sms_spam dataset
from datasets import load_dataset
raw_dataset = load_dataset("sms_spam")

# check if mps gpu is available
torch.backends.mps.is_available()
torch.backends.mps.is_built()
mps_device = torch.device("mps")
x = torch.ones(45, device="mps")

Found cached dataset sms_spam (/Users/jayreddy/.cache/huggingface/datasets/sms_spam/plain_text/1.0.0/53f051d3b5f62d99d61792c91acefe4f1577ad3e4c216fb0ad39e30b9f20019c)
100%|███████████████████████████████████| 1/1 [00:00<00:00, 394.02it/s]


In [25]:
def preprocessing_tokenize_data(sms):
    return tokenizer(
        sms['sms'], padding='max_length', truncation=True, max_length=128
    )

In [27]:
# final preprocessing steps after tokenizing dataset
tokenized_dataset = raw_dataset.map(preprocessing_tokenize_data, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns('sms')
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")
tokenized_dataset = tokenized_dataset.with_format("torch")
tokenized_dataset['train'].features

Loading cached processed dataset at /Users/jayreddy/.cache/huggingface/datasets/sms_spam/plain_text/1.0.0/53f051d3b5f62d99d61792c91acefe4f1577ad3e4c216fb0ad39e30b9f20019c/cache-8021552568c79186.arrow


{'labels': ClassLabel(names=['ham', 'spam'], id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [29]:
from transformers import TrainingArguments
training_arguments = TrainingArguments(output_dir="training_checkpoints")